In [1]:
data_var = '2024-02-01'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10612,2024-02-01,Eua Nba,00:00,Portland Trail Blazers,Milwaukee Bucks,5.04,1.19,236.5,1.88,2.04,10.5,2.01,1.93,2wII32e8,0.198413,0.840336,0.531915,0.490196,0.038749,0.6,0.2,NaN,NaN,583.734,272.051076,0.466053,174.772,49.780703,0.284832,424.786,175.886,510.90,260.01,1.0,0.0,0.0,0.0,0.873952,0.057723,0.028715,7.12,1.424,2.837079,0.387507,0.6,0.212493,-2.71,-0.542,-0.350554,0.671871,0.5,-0.171871
10613,2024-02-01,Europa Euroliga,16:00,Alba Berlin,Valencia,2.28,1.67,156.5,1.83,1.95,1.5,2.10,1.67,KxsvoI5f,0.438596,0.598802,0.546448,0.512821,0.037399,0.6,0.2,NaN,NaN,301.212,129.566784,0.430151,212.578,127.202592,0.598381,209.206,259.410,234.96,197.12,0.0,0.0,0.0,0.0,0.218398,0.044896,0.161303,-2.36,-0.472,-2.711864,0.389100,0.3,-0.089100,4.60,0.920,0.728261,0.000000,0.0,0.000000
10614,2024-02-01,Europa Euroliga,16:30,Baskonia,Maccabi Tel Aviv,1.73,2.18,172.5,1.91,1.91,-3.5,2.00,1.71,INaspxL0,0.578035,0.458716,0.523560,0.523560,0.036750,0.4,0.6,NaN,NaN,192.116,67.958538,0.353737,227.534,104.968701,0.461332,132.440,248.556,93.62,412.08,0.0,0.0,0.0,0.0,0.162761,0.000000,0.110545,0.40,0.080,9.125000,0.000000,0.0,0.000000,-5.00,-1.000,-1.180000,0.000000,0.0,0.000000
10615,2024-02-01,Europa Euroliga,16:30,Bayern,Olympiakos,1.87,1.95,156.5,1.91,1.91,-2.5,2.10,1.67,Cpaoqdz7,0.534759,0.512821,0.523560,0.523560,0.047580,0.6,0.8,NaN,NaN,202.296,113.448543,0.560805,235.376,203.246366,0.863497,158.406,233.548,200.48,595.00,0.0,1.0,0.0,0.0,0.029617,0.000000,0.161303,-1.70,-0.340,-2.558824,0.000000,0.0,0.000000,-2.29,-0.458,-2.074236,0.000000,0.0,0.000000
10616,2024-02-01,Europa Euroliga,16:45,Real Madrid,Lyon-Villeurbanne,1.07,9.60,166.5,1.81,1.95,-16.5,2.01,1.78,jybkrGkD,0.934579,0.104167,0.552486,0.512821,0.038746,0.8,0.4,NaN,NaN,123.776,14.035323,0.113393,209.998,71.520858,0.340579,124.246,378.978,133.56,285.76,0.0,0.0,0.0,0.0,1.130576,0.052657,0.085823,1.06,0.212,0.330189,0.833480,1.0,0.166520,-1.24,-0.248,-34.677419,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10710,2024-02-01,Rússia Premier League Feminina,05:30,Enisey Krasnoyarsk F,Nika Siktivkar F,2.32,1.56,145.5,1.88,1.79,2.5,1.99,1.73,AJqxrgnL,0.431034,0.641026,0.531915,0.558659,0.072060,0.0,0.0,NaN,NaN,207.506,171.904184,0.828430,121.556,41.125879,0.338329,0.000,0.000,90.86,99.12,0.0,0.0,0.0,0.0,0.277011,0.034681,0.098843,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10711,2024-02-01,Suécia Basketligan,15:04,BC Lulea,Borås,2.03,1.71,169.5,1.82,1.88,1.5,1.88,1.82,UuCWjOkf,0.492611,0.584795,0.549451,0.531915,0.077406,0.4,1.0,NaN,NaN,167.244,55.120955,0.329584,142.922,75.294603,0.526823,151.306,168.082,247.50,126.90,0.0,0.0,0.0,0.0,0.121002,0.022933,0.022933,1.48,0.296,3.479730,0.000000,0.0,0.000000,0.64,0.128,5.546875,0.000000,0.0,0.000000
10712,2024-02-01,Suécia Basketligan,15:04,Jamtland,Koping Stars,1.12,5.56,160.5,1.86,1.84,-13.5,2.06,1.68,O2Bzjr50,0.892857,0.179856,0.537634,0.543478,0.072713,0.2,0.6,NaN,NaN,130.034,27.211781,0.209267,377.986,173.167994,0.458133,121.812,422.972,130.05,487.62,0.0,0.0,0.0,0.0,0.939986,0.007644,0.143690,-2.40,-0.480,-0.250000,0.000000,0.0,0.000000,-3.30,-0.660,-6.909091,0.000000,0.0,0.000000
10713,2024-02-01,Rússia Premier League Feminina,12:00,Samara F,Neftyanik Omsk 

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Portland Trail Blazers,Milwaukee Bucks,236.5,1.88,1.0,Over
